## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature that would be acceptable on your vacation (F)? "))
max_temp = float(input("What is the maximum temperature that would be acceptable on your vacation (F)? "))

What is the minimum temperature that would be acceptable on your vacation (F)? 75
What is the maximum temperature that would be acceptable on your vacation (F)? 90


In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kapaa,22.0752,-159.3190,78.78,69,40,9.22,US,11/27/2021 4:48
1,1,Mahebourg,-20.4081,57.7000,80.89,65,20,10.36,MU,11/27/2021 4:48
3,3,Porbandar,21.6422,69.6093,75.18,69,7,6.91,IN,11/27/2021 4:48
11,11,Rikitea,-23.1203,-134.9692,76.17,73,100,15.32,PF,11/27/2021 4:48
13,13,Hithadhoo,-0.6000,73.0833,81.25,71,79,13.80,MV,11/27/2021 4:47
17,17,Sur,22.5667,59.5289,80.01,54,73,13.33,OM,11/27/2021 4:48
20,20,Bubaque,11.2833,-15.8333,82.96,71,99,6.13,GW,11/27/2021 4:48
30,30,Manaure,11.7751,-72.4445,77.97,85,95,7.81,CO,11/27/2021 4:48
33,33,Georgetown,5.4112,100.3354,87.73,70,20,3.44,MY,11/27/2021 4:47
36,36,Champasak,14.7500,106.0000,87.42,43,98,4.36,LA,11/27/2021 4:48


In [25]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       152
City          152
Lat           152
Lng           152
Max Temp      152
Humidity      152
Cloudiness    152
Wind Speed    152
Country       152
Date          152
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kapaa,22.0752,-159.3190,78.78,69,40,9.22,US,11/27/2021 4:48
1,1,Mahebourg,-20.4081,57.7000,80.89,65,20,10.36,MU,11/27/2021 4:48
3,3,Porbandar,21.6422,69.6093,75.18,69,7,6.91,IN,11/27/2021 4:48
11,11,Rikitea,-23.1203,-134.9692,76.17,73,100,15.32,PF,11/27/2021 4:48
13,13,Hithadhoo,-0.6000,73.0833,81.25,71,79,13.80,MV,11/27/2021 4:47
...,...,...,...,...,...,...,...,...,...,...
596,596,Kollam,8.8833,76.6000,84.07,71,87,5.48,IN,11/27/2021 5:43
598,598,Port Blair,11.6667,92.7500,84.04,71,32,10.78,IN,11/27/2021 5:47
599,599,Nioro,13.3500,-15.7500,75.78,42,98,6.13,GM,11/27/2021 5:47
604,604,Malinyi,-8.9333,36.1333,75.29,75,87,2.55,TZ,11/27/2021 5:47


In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kapaa,US,78.78,22.0752,-159.3190,
1,Mahebourg,MU,80.89,-20.4081,57.7000,
3,Porbandar,IN,75.18,21.6422,69.6093,
11,Rikitea,PF,76.17,-23.1203,-134.9692,
13,Hithadhoo,MV,81.25,-0.6000,73.0833,
17,Sur,OM,80.01,22.5667,59.5289,
20,Bubaque,GW,82.96,11.2833,-15.8333,
30,Manaure,CO,77.97,11.7751,-72.4445,
33,Georgetown,MY,87.73,5.4112,100.3354,
36,Champasak,LA,87.42,14.7500,106.0000,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{City Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

****************************EDITED THIS LINE CITY DESCRIPTION

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig